In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from i3cols_dataloader import load_events


from tqdm import tqdm

import dragoman as dm

from likelihood import LLH

%load_ext line_profiler

In [2]:
plt.rcParams['figure.figsize'] = [12., 8.]
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14 
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 14

# Test NNs
Using an independent MC file

In [3]:
events, labels = load_events(dir='/home/iwsatlas1/peller/work/oscNext/level7_v01.04/140000_i3cols_test',
                             recos = {'spefit': 'SPEFit2_DC',
                                    'retro' : 'retro_crs_prefit__median__neutrino'})

In [4]:
events[0]

{'total_charge': 6.1000004,
 'hits': array([[-3.2960e+01,  6.2440e+01, -2.5310e+02,  1.0289e+04,  9.2500e-01,
          0.0000e+00,  0.0000e+00,  4.4000e+01,  4.4000e+01],
        [-9.6800e+00, -7.9500e+01, -4.1573e+02,  1.0814e+04,  7.7500e-01,
          0.0000e+00,  0.0000e+00,  8.4000e+01,  4.7000e+01],
        [-1.0970e+01,  6.7200e+00, -2.2008e+02,  1.1105e+04,  8.7500e-01,
          0.0000e+00,  0.0000e+00,  8.5000e+01,  1.9000e+01],
        [-1.0970e+01,  6.7200e+00, -2.4110e+02,  9.9380e+03,  6.2500e-01,
          1.0000e+00,  1.0000e+00,  8.5000e+01,  2.2000e+01],
        [-1.0970e+01,  6.7200e+00, -2.4110e+02,  1.0024e+04,  1.1250e+00,
          1.0000e+00,  1.0000e+00,  8.5000e+01,  2.2000e+01],
        [-1.0970e+01,  6.7200e+00, -2.5512e+02,  9.8820e+03,  5.7500e-01,
          1.0000e+00,  1.0000e+00,  8.5000e+01,  2.4000e+01],
        [-1.0970e+01,  6.7200e+00, -2.6213e+02,  9.8930e+03,  6.7500e-01,
          1.0000e+00,  1.0000e+00,  8.5000e+01,  2.5000e+01],
        [-1.

In [5]:
# import pickle

# with open('test_data/test_events.pkl', 'wb') as f:
#     pickle.dump(events, f, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
freedom_llh = LLH(hitnet_file='models/HitNet_logE18_May_2020-21h02/epoch_42_model.hdf5',
          chargenet_file='models/ChargeNet_logE_18_May_2020-21h03/epoch_400_model.hdf5',
          epsilon=1e-7,
         )

In [7]:
from spherical_opt import spherical_opt

In [8]:
event = events[123]

In [19]:
N_points = 9

initial_points = np.repeat(event['spefit'][np.newaxis, :], N_points, axis=0)

In [20]:
noise = np.random.randn(N_points, 8)

noise[:, 6:] = (noise[:, 6:] + 10) * 3
noise[:, 0:4] *= 50
noise[:, 4] *= 3
noise[:, 4:6] *= 6
noise[:, 4] = noise[:, 4] % 2*np.pi
noise[:, 5] = noise[:, 5] % np.pi

In [21]:
freedom_llh(event, event['params'])[0][0]

-25.414892

In [22]:
def fun(x):
    return freedom_llh(event, x)[0][0]

In [23]:
seed = initial_points + noise
seed[:, 6:] = np.clip(seed[:, 6:], 0, 1e5)

In [24]:
opt = spherical_opt.spherical_opt(fun, method='Nelder-Mead', initial_points=seed, spherical_indices=[(4,5)], meta=True, fstdthresh=0.1)

In [25]:
event['retro']

array([ 9.8726997e+01, -4.6753323e+01, -4.3313397e+02,  9.7726455e+03,
        2.8385775e+00,  1.3605294e+00,  2.5890183e-01,  2.0623093e+00],
      dtype=float32)

In [26]:
for l,t,s,r,f in zip(labels, event['params'], event['spefit'], event['retro'], opt['x']):
    print(f'{l.ljust(20)}{t:.2f}\t{s:.1f}\t{r:.1f}\t{f:.1f}')

x                   80.09	42.4	98.7	261.1
y                   -45.76	-53.2	-46.8	-101.2
z                   -429.46	-383.4	-433.1	-472.1
time                9729.45	9107.9	9772.6	9293.5
azimuth             2.74	4.5	2.8	6.1
zenith              1.29	0.8	1.4	1.9
cascade_energy      0.59	0.0	0.3	25.9
track_energy        2.30	0.0	2.1	23.2


In [27]:
opt

{'stopping_flag': 1,
 'n_calls': 133,
 'nit': 81,
 'method': 'Nelder-Mead',
 'fun': -16.16634750366211,
 'x': array([ 2.61127844e+02, -1.01201118e+02, -4.72144664e+02,  9.29353907e+03,
         6.06000471e+00,  1.87869430e+00,  2.59363168e+01,  2.32178617e+01]),
 'final_simplex': [array([[ 2.60518364e+02, -9.35321247e+01, -4.61134263e+02,
           9.28319501e+03,  6.07270956e+00,  1.69551587e+00,
           2.58321982e+01,  2.36877705e+01],
         [ 2.64022813e+02, -9.59061280e+01, -4.65322893e+02,
           9.29236483e+03,  6.01862240e+00,  1.85070825e+00,
           2.54866117e+01,  2.34714291e+01],
         [ 2.58402964e+02, -1.03989340e+02, -4.65325800e+02,
           9.27999996e+03,  5.99525833e+00,  1.75027704e+00,
           2.62813575e+01,  2.36571814e+01],
         [ 2.61127844e+02, -1.01201118e+02, -4.72144664e+02,
           9.29353907e+03,  6.06000471e+00,  1.87869430e+00,
           2.59363168e+01,  2.32178617e+01],
         [ 2.59000370e+02, -1.03989283e+02, -4.71034